In [ ]:
# Extrator de Dados
import numpy as np
import time
import requests
import pandas as pd
import logging
from datetime import datetime, timezone
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import display



# Definição do intervalo de datas para coleta de dados
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 4, 15)

# Detalhes da requisição da API
url_events = "https://api.b365api.com/v3/events/ended"
url_odds = "https://api.b365api.com/v2/event/odds"
token = "183604-pWN7flhoAsWGu8"  # Substitua pelo seu token real
league_ids = [22614, 23114, 37298, 38439]  # Lista de ligas a serem coletadas

# Função genérica para requisições com retries
def make_request_with_retry(url, params, retries=5, backoff_factor=2, timeout=10):
    for attempt in range(1, retries + 1):
        try:
            response = requests.get(url, params=params, timeout=timeout)
            response.raise_for_status()  # Levanta exceção se status_code >= 400
            return response
        except requests.RequestException as e:
            wait_time = backoff_factor ** attempt  # Exponential backoff
            logging.warning(f"Tentativa {attempt} falhou: {e}. Retentando em {wait_time} segundos.")
            time.sleep(wait_time)
            if attempt == retries:
                logging.error(f"Todas as {retries} tentativas falharam para URL: {url}")
                raise e

# Função para buscar eventos em uma data e liga específicas
def fetch_events_for_date(date, league_id):
    formatted_date = date.strftime('%Y%m%d')
    params = {
        "token": token,
        "sport_id": "1",
        "league_id": str(league_id),
        "day": formatted_date,
        "page": 1
    }
    events = []
    while True:
        try:
            response = make_request_with_retry(url_events, params)
            data = response.json()
            current_events = data['results']
            # Adiciona league_id a cada evento
            for event in current_events:
                event['league_id'] = league_id
            events.extend(current_events)
            if params['page'] * data['pager']['per_page'] < data['pager']['total']:
                params['page'] += 1
            else:
                break
        except Exception as e:
            logging.error(f"Erro ao buscar eventos para a data {formatted_date} e liga {league_id}: {e}")
            break
    return events

# Função para pegar os dados da API de odds
def fetch_odds_for_event(event_id):
    params = {
        'event_id': event_id,
        'token': token
    }
    try:
        response = make_request_with_retry(url_odds, params)
        data = response.json()
        if 'success' in data and data['success'] == 1:
            results = data.get('results', {})
            odds = results.get('odds', {})

            markets = ['1_1', '1_2', '1_3']
            filtered_odds = {}

            for market in markets:
                market_odds = odds.get(market, [])
                selected_odd = None
                min_add_time = float('inf')

                for odd in market_odds:
                    add_time = int(odd.get('add_time'))
                    ss = odd.get('ss')

                    if ss == '0-0' and add_time < min_add_time:
                        min_add_time = add_time
                        selected_odd = odd

                if not selected_odd:
                    max_add_time = float('-inf')
                    for odd in market_odds:
                        add_time = int(odd.get('add_time'))
                        ss = odd.get('ss')

                        if ss is None and add_time > max_add_time:
                            max_add_time = add_time
                            selected_odd = odd

                if selected_odd:
                    if market == '1_3':
                        filtered_odds[market] = {
                            'over_od': selected_odd.get('over_od'),
                            'under_od': selected_odd.get('under_od'),
                            'handicap': selected_odd.get('handicap'),
                            'add_time': datetime.fromtimestamp(int(selected_odd.get('add_time')), tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
                        }
                    else:
                        filtered_odds[market] = {
                            'home_od': selected_odd.get('home_od'),
                            'draw_od': selected_odd.get('draw_od'),
                            'away_od': selected_odd.get('away_od'),
                            'handicap': selected_odd.get('handicap'),
                            'add_time': datetime.fromtimestamp(int(selected_odd.get('add_time')), tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
                        }
                else:
                    filtered_odds[market] = None
            return filtered_odds
    except Exception as e:
        logging.error(f"Erro ao buscar odds para o evento {event_id}: {e}")
    return None

grid_start = datetime(2025, 5, 13)
grid_end = datetime(2025, 5, 14)

for date in pd.date_range(grid_start, grid_end):
    all_events = []
    for single_date in pd.date_range(date, date):
        for league_id in league_ids:
            events = fetch_events_for_date(single_date, league_id)
            all_events.extend(events)
            print(f"Coletados {len(events)} eventos para liga {league_id} em {single_date.strftime('%Y-%m-%d')}")

    # Criando o DataFrame com os eventos coletados
    df_events = pd.DataFrame([{
        'event_id': event['id'],
        'league': event['league_id'],  # Nova coluna adicionada
        'date': datetime.utcfromtimestamp(int(event['time'])).strftime('%Y-%m-%d %H:%M:%S'),
        'away_team': event['away']['name'].split('(')[0].strip(),
        'away_player': event['away']['name'].split('(')[1].split(')')[0] if '(' in event['away']['name'] else '',
        'away_score': int(event.get('scores', {}).get('2', {}).get('away', 0)),  # Convertendo para inteiro
        'home_team': event['home']['name'].split('(')[0].strip(),
        'home_player': event['home']['name'].split('(')[1].split(')')[0] if '(' in event['home']['name'] else '',
        'home_score': int(event.get('scores', {}).get('2', {}).get('home', 0)),  # Convertendo para inteiro
    } for event in all_events])

    # Conversão de 'gols_casa', 'gols_fora', 'corners_casa' e 'corners_fora' para numérico e remoção de linhas com NaN
    df_events['home_score'] = pd.to_numeric(df_events['home_score'].replace('N/A', np.nan), errors='coerce')
    df_events['away_score'] = pd.to_numeric(df_events['away_score'].replace('N/A', np.nan), errors='coerce')

    df_events['total_score'] = df_events['home_score'] + df_events['away_score']

    # Função para processar cada linha para odds
    def process_event_odds(index, row):
        event_id = row['event_id']
        logging.info(f"Buscando odds para event_id: {event_id}")
        try:
            odds_data = fetch_odds_for_event(event_id)
            resultado = {}

            if odds_data:
                for market, odds in odds_data.items():
                    if odds:
                        if market == '1_3':
                            resultado[f'{market}_over_od'] = odds['over_od']
                            resultado[f'{market}_under_od'] = odds['under_od']
                            resultado[f'{market}_handicap'] = odds['handicap']
                        else:
                            resultado[f'{market}_home_od'] = odds['home_od']
                            resultado[f'{market}_draw_od'] = odds['draw_od']
                            resultado[f'{market}_away_od'] = odds['away_od']
                            resultado[f'{market}_handicap'] = odds['handicap']
                            
                        resultado[f'{market}_add_time'] = odds['add_time']

            return index, resultado
        except Exception as e:
            logging.error(f"Erro ao buscar odds para o evento {event_id}: {e}")
            return index, {}

    # Adicionando odds ao DataFrame de eventos
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(process_event_odds, index, row) for index, row in df_events.iterrows()]
        for future in as_completed(futures):
            index, odds_result = future.result()
            for col, value in odds_result.items():
                df_events.loc[index, col] = value
            logging.info(f"Odds adicionadas para event_id no índice {index}")

    # Exibição dos primeiros registros para verificação
    # display(df_events.head())


    # Salvamento dos dados
    df_events.to_csv(f'odds_data/{date:%d-%m}.csv', index=False)
    print(f"Processamento concluído e dados salvos para data {date:%d-%m}.")

Coletados 266 eventos para liga 22614 em 2025-05-10
Coletados 267 eventos para liga 23114 em 2025-05-10
Coletados 203 eventos para liga 37298 em 2025-05-10
Coletados 280 eventos para liga 38439 em 2025-05-10


C:\Users\Enzo\AppData\Local\Temp\ipykernel_8668\1873793678.py:144: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'date': datetime.utcfromtimestamp(int(event['time'])).strftime('%Y-%m-%d %H:%M:%S'),


Processamento concluído e dados salvos para data 10-05.
Coletados 218 eventos para liga 22614 em 2025-05-11
Coletados 280 eventos para liga 23114 em 2025-05-11
Coletados 203 eventos para liga 37298 em 2025-05-11
Coletados 280 eventos para liga 38439 em 2025-05-11


C:\Users\Enzo\AppData\Local\Temp\ipykernel_8668\1873793678.py:144: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'date': datetime.utcfromtimestamp(int(event['time'])).strftime('%Y-%m-%d %H:%M:%S'),


Processamento concluído e dados salvos para data 11-05.
Coletados 334 eventos para liga 22614 em 2025-05-12
Coletados 299 eventos para liga 23114 em 2025-05-12
Coletados 212 eventos para liga 37298 em 2025-05-12
Coletados 280 eventos para liga 38439 em 2025-05-12


C:\Users\Enzo\AppData\Local\Temp\ipykernel_8668\1873793678.py:144: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  'date': datetime.utcfromtimestamp(int(event['time'])).strftime('%Y-%m-%d %H:%M:%S'),


Processamento concluído e dados salvos para data 12-05.
